<a href="https://colab.research.google.com/github/perlatomdpi/NLP/blob/main/BERT_Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BERT Tokenizer**
Example of tokenization using BERT



# **Import dependencies**


In [ ]:
import numpy as np
import math
import re # advanced text pre-processing
import pandas as pd
